In [ ]:
import os 
os.chdir("../")

In [8]:
pwd

'c:\\Ankit\\cd\\fastApi\\Medical-bot'

In [ ]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [2]:
def load_pdf(data):
    loader = DirectoryLoader(
        data,
        glob = "*.pdf",
        loader_cls=PyPDFLoader
    )
    documents = loader.load()
    return documents

In [10]:
extracted_text = load_pdf("data")
len(extracted_text)

8

In [11]:
extracted_text

[Document(metadata={'producer': 'Microsoft® Word for Microsoft 365', 'creator': 'Microsoft® Word for Microsoft 365', 'creationdate': '2025-07-29T21:19:21+05:30', 'author': 'cmpn- staff 2', 'moddate': '2025-07-29T21:19:21+05:30', 'source': 'data\\a.pdf', 'total_pages': 8, 'page': 0, 'page_label': '1'}, page_content='Experiment 01 \nLearning Objective: Student should be able to apply Assembly Language Programing to enter \nand display 8 bit & 16 bits number  \n \nTools: TASM/MASM \n \n Theory:  \nAssembler Directives :- \nThis type of statements includes commands that are addressed to the assembler, such as: \nConstant and variable definition. \nAllocation of memory space and initialization of memory, and \nControl of the assembly process \n \nList of assembler directives \na. Data Allocation Directives \nDB…………..define byte \nDW………….define word (2 bytes) \nDD………….define double word (4 bytes) \nDQ………….define quadword (8 bytes) \nDT…………..define tenbytes  \nEQU…………equate, assign numeric ex

In [ ]:
from typing import List
from langchain.schema import Document
def filter_to_minimal_docs (docs: List [Document]) -> List [Document]:
    """
    Given a list of Document objects, return a new list of Document objects
    containing only 'source' in metadata and the original page content.
    """
    minimal_docs: List[Document] = []
    for doc in docs:
        src = doc.metadata.get("source")
        minimal_docs.append( Document(
            page_content=doc.page_content,
            metadata={"source": src}
            ))
    return minimal_docs

In [16]:
minimal_docs=filter_to_minimal_docs(extracted_text)
minimal_docs

[Document(metadata={'source': 'data\\a.pdf'}, page_content='Experiment 01 \nLearning Objective: Student should be able to apply Assembly Language Programing to enter \nand display 8 bit & 16 bits number  \n \nTools: TASM/MASM \n \n Theory:  \nAssembler Directives :- \nThis type of statements includes commands that are addressed to the assembler, such as: \nConstant and variable definition. \nAllocation of memory space and initialization of memory, and \nControl of the assembly process \n \nList of assembler directives \na. Data Allocation Directives \nDB…………..define byte \nDW………….define word (2 bytes) \nDD………….define double word (4 bytes) \nDQ………….define quadword (8 bytes) \nDT…………..define tenbytes  \nEQU…………equate, assign numeric expression to a name \n    Examples:3 \n          db         100 dup (?)      define 100 bytes, with no initial values for bytes \n          db        “Hello” define 5 bytes, ASCII equivalent of “Hello”. \n          maxint equ 327673 \n          count equ 10 

In [ ]:
def text_split(minimal_docs):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 500,
        chunk_overlap = 20
    )

    text_chunk = text_splitter.split_documents(minimal_docs)
    return texts_chunk

In [21]:
text_chunks = text_split(minimal_docs)
len(text_chunks)

22

In [24]:
from langchain.embeddings import HuggingFaceEmbeddings
def download_embeddings():
    """
    Download and return the HuggingFace embeddings model.
    """
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
    embeddings = HuggingFaceEmbeddings(
        model_name= model_name
        )
    return embeddings
embedding = download_embeddings()

In [25]:
embedding

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [26]:
vector = embedding.embed_query("Hello")
vector

[-0.06277173012495041,
 0.0549587644636631,
 0.052164845168590546,
 0.08579001575708389,
 -0.0827489048242569,
 -0.07457298785448074,
 0.06855474412441254,
 0.018396401777863503,
 -0.08201131224632263,
 -0.03738486021757126,
 0.01212488953024149,
 0.003518301760777831,
 -0.004134277813136578,
 -0.043784454464912415,
 0.021807288751006126,
 -0.005102697294205427,
 0.019546590745449066,
 -0.04234873875975609,
 -0.11035966873168945,
 0.005424531176686287,
 -0.05573474243283272,
 0.028052419424057007,
 -0.023158712312579155,
 0.028481345623731613,
 -0.05370963364839554,
 -0.052601564675569534,
 0.033939216285943985,
 0.04538865014910698,
 0.0237184539437294,
 -0.0731208324432373,
 0.054777760058641434,
 0.017047306522727013,
 0.08136036992073059,
 -0.00286271795630455,
 0.011958097107708454,
 0.07355853170156479,
 -0.09423743933439255,
 -0.0813620537519455,
 0.040015410631895065,
 0.0006922061438672245,
 -0.013393313623964787,
 -0.05453810095787048,
 0.005151402670890093,
 -0.0261398162692

In [27]:
len(vector)

384

In [28]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [29]:
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY") 

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY

In [30]:
from pinecone import Pinecone
pc = Pinecone(api_key = PINECONE_API_KEY )
pc

In [33]:
from pinecone import ServerlessSpec
index_name = "medical-chatbot"
if not pc.has_index(index_name):
    pc.create_index(
        name = index_name,
        dimension=384, 
        metric="cosine" ,# Cosine similarity
        spec=ServerlessSpec (cloud="aws", region="us-east-1")
)
index = pc.Index(index_name)

In [34]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents = text_chunks,
    embedding = embedding,
    index_name = index_name
)

In [35]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_existing_index(
    embedding = embedding,
    index_name = index_name
)

In [39]:
random_data = Document(
    page_content = "I hate coding",
    metadata = {"source":"googly"}
)

In [40]:
docsearch.add_documents([random_data])

['acb65334-4ed0-4492-9c3c-9db457edc62e']

In [102]:
retriever = docsearch.as_retriever(search_type="similarity",search_kwargs={"k":3})
retrieved_doc = retriever.invoke("What is procedure")
retrieved_doc

[Document(id='4f185146-33bb-4be8-9412-31127bff4546', metadata={'source': 'data\\a.pdf'}, page_content='shown below: \n__ g ; complete execution of program in single step. \n__ t ; Stepwise execution. \n__d ds: starting address or ending address ; To see data in memory locations \n__p; Used to execute interrupt or procedure during stepwise execution of program \n__ q ; To quit the execution. \n \n5. Procedure/ Algorithm \nProgram to accept 8 bit number and display 8 bit number  \nExplanation: Conversions from ASCII to binary usually start with keyboard data entry. If a single'),
 Document(id='84714dd0-d5d8-4b90-93a9-268076bfe5e3', metadata={'source': 'data\\a.pdf'}, page_content='ENDS…………used to indicate the end of the segment. \nEND…………. used to indicate the end of program. \nPROC……........used to indicate the beginning of a procedure. \nENDP……….. .used to indicate the end of a program.  \nENDM………. used to indicate the end of a program.  \nSEGMENT…...used to indicate the start of the s

In [115]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI

chatModel = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    google_api_key=os.getenv("GEMINI_API_KEY")
  )


In [117]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("Answer based on context:\n{context}\n\nQuestion: {input}")
document_chain = create_stuff_documents_chain(chatModel, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)


In [ ]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "If you gets the related content then try to brief it more in detail adding more info."
    "Use the retrieved context to answer the question as best as possible. "
    "If the context seems partially relevant, try to summarize it. "
    "If you still don’t find anything useful, then say 'I don’t know.' "
    "\n\nContext:\n{context}"
)

prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt),
    ("human", "{input}"),
    ]
)

In [120]:
qa = create_stuff_documents_chain(chatModel,prompt)
rag_chain = create_retrieval_chain(retriever , qa)

In [123]:
resp = rag_chain.invoke({ "input": "full code for displaying 16-bit number" })
print(resp["answer"])
   

The provided text gives snippets of 8086 assembly code for displaying numbers, but it's incomplete and lacks crucial parts for a full 16-bit number display.  It shows how to display a single character (low nibble) and mentions printing a prompt, but doesn't show how to handle the entire 16-bit number.  To display a 16-bit number, you need to break it down into individual digits (or nibbles), convert them to their ASCII representations, and then display each character sequentially.

Here's a more complete example, though it still assumes input is restricted to digits 0-9 (not handling A-F for hexadecimal):


```assembly
.model small
.stack 100h

.data
msg1 dw 10,13,"Enter 16-bit number (0-9 only): $"
msg2 dw 10,13,"The 16-bit number is: $"
num dw ? ; Variable to store the 16-bit number

.code
main proc
    mov ax, @data
    mov ds, ax

    ; Display prompt message
    mov ah, 09h
    lea dx, msg1
    int 21h

    ; Read the 16-bit number (this part needs significant improvement for robu